# Parameter example

## 1. Setup

 Setup notebook:

In [1]:
# just for the notebook setup so we have the ami_sim module
import sys
sys.path.append('../')

First we import from ami_sim

In [2]:
from ami_sim_mtl.core.instrument import constants
from ami_sim_mtl.core.core import param_functions

All codes should have the following variables (at the start of the code)

In [3]:
# =============================================================================
# Define variables
# =============================================================================
# set name
__NAME__ = 'bin/test.py'
__DESCRIPTION__ = 'test script for AMI_SIM_MTL'
# get default constants
consts = constants.Consts
# copy for update
lconsts = consts.copy(__NAME__)
# set very basic constants
__VERSION__ = lconsts.constants['PACKAGE_VERSION'].value
__DATE__ = lconsts.constants['PACKAGE_VERSION_DATE'].value

Define custom arguments (just for this code)

Example here we add a "scene" argument with the following properties:
1. name = "SCENE"          # parameter name = "SCENE"
2. dtype = str             # data type = python string
3. source = `__NAME__`     # where the parameter was defined (here `__NAME__` is the code name)
4. user = True             # whether to populate this value in any config file generated
5. argument = True         # whether to use this variable from command line (must define "command" in this case)
6. group = 'code'          # which group this variable belongs to (mostly for config file)
7. description = ...       # a description for the help file / config file
8. command = `['--scene']` # a list of commands to be able to use at run time e.g. in this case --scene=test.fits

In [4]:
# Define the scene fits file
lconsts.add_argument('SCENE', value=None, dtype=str,
                     source=__NAME__, user=True, argument=True,
                     group='code', description='Define the scene fits file',
                     command=['--scene'])

Now we can get parameters:

In [5]:
kwargs = dict()                   # this is used for function calls (ignore for now)
sys.argv = 'test.py'.split()      # this is where command lines arguments come from
params = param_functions.setup(lconsts, kwargs, description=__DESCRIPTION__)

For example params currently has the following:

In [6]:
params

ParamDict:

 	USER_CONFIG_FILE:             None                                          # core.instruments.constants.py
 	PACKAGE_NAME:                 ami_sim_mtl                                   # core.instruments.constants.py
 	PACKAGE_VERSION:              0.0.001                                       # core.instruments.constants.py
 	PACKAGE_VERSION_DATE:         2020-05-21                                    # core.instruments.constants.py
 	PACKAGE_DIRECTORY:            ami_sim_data                                  # core.instruments.constants.py
 	GENERATE_CONFIG_FILE:         False                                         # core.instruments.constants.py
 	DIRECTORY:                    /home/cook/ami_sim_data                       # core.core.constant_functions.py._read_from_config_file()
 	ENV_DIR:                      AMIDIR                                        # core.instruments.constants.py
 	ADD_JITTER:                   True                                          # c

Let test this again with some arguments from the command line as follows:

In [7]:
sys.argv = 'test.py --scene=test.fits --add_jitter=False'.split()

In [8]:
kwargs = dict()                   # this is used for function calls
params = param_functions.setup(lconsts, kwargs, description=__DESCRIPTION__)

In [9]:
params

ParamDict:

 	USER_CONFIG_FILE:             None                                          # core.instruments.constants.py
 	PACKAGE_NAME:                 ami_sim_mtl                                   # core.instruments.constants.py
 	PACKAGE_VERSION:              0.0.001                                       # core.instruments.constants.py
 	PACKAGE_VERSION_DATE:         2020-05-21                                    # core.instruments.constants.py
 	PACKAGE_DIRECTORY:            ami_sim_data                                  # core.instruments.constants.py
 	GENERATE_CONFIG_FILE:         False                                         # core.instruments.constants.py
 	DIRECTORY:                    /home/cook/ami_sim_data                       # core.core.constant_functions.py._read_from_config_file()
 	ENV_DIR:                      AMIDIR                                        # core.instruments.constants.py
 	ADD_JITTER:                   False                                         # s

Note the difference between the value of `ADD_JITTER` and `SCENE` from the previous example, the source (third column above has also changed -- to `sys.argv`)

## 2. Generating a config file

There is a special mode for any code using the param_fucntions.setup function.

One can generate a config file with the current settings given (from command line / another config file / the constants file)

This is done using the argument `--getconfig=True` (the default value is `--getconfig=False`)

In [10]:
# set the command line arguments (for notebook only)
sys.argv = 'test.py --getconfig True'.split()

In [16]:
kwargs = dict()                   # this is used for function calls

try:
    params = param_functions.setup(lconsts, kwargs, description=__DESCRIPTION__)
# just so the notebook continues after this
except Exception as e:
    print('Error {0}: {1}'.format(e.__name__, e))

Writing constants file to /home/cook/ami_sim_data/config/None/None


AttributeError: 'FileNotFoundError' object has no attribute '__name__'

As you can see above this added a file to `./outputs/` called `user_config.ini`

This file can be called from the command line using the `--config` argument.

## 3. The help file

As mentioned above the descriptions we gave of `SCENE` will appear in the help file.

The help file is accessed as always through `-h` or `--help`. 

We can emulate this here with `sys.argv`:

In [ ]:
# set the command line arguments (for notebook only)
sys.argv = 'test.py --help'.split()

which will run when we run the `param_functions.setup` function:

In [ ]:
kwargs = dict()                   # this is used for function calls
# ignore the try/except statement here it is just because the -h/--help argument will force an exit of python
try:
    params = param_functions.setup(lconsts, kwargs, description=__DESCRIPTION__)
except SystemExit:
    pass

## 4. Use in a code

We put the main code we with to run in a `__main__()` sub function so we can log and manage exception that come from the code. When we call the code we use the `main()` function that will run the parameter setup and handle any errors from our `__main__()` function. (By handle here I mean deal with exceptions and log/shut things done in a good way).

A code using a good setup would look as follows:

In [ ]:
def main(**kwargs):
    # get params (run time + config file + constants file)
    params = param_functions.setup(lconsts, kwargs,
                                   description=__DESCRIPTION__)
    # run the __main__ to return products
    if not params['GENERATE_CONFIG_FILE']:
        # note eventually this will be a call to a function which manages exceptions
        return __main__(params)


def __main__(params):
    # main code here
    print('Hello, World')
    print('My code goes here')
    return params
    
    

This is then called from the main code (or from an import) as follows:

In [ ]:
# For the note book we need to set the command line arguments
sys.argv = 'test.py'.split()
# run main code
ll = main()


Note to import this it would look as follows:


```
import paramtest

test.main()
```


where any parameters can be defined in the `.main()` call i.e.:

In [ ]:
# just for the notebook
sys.path.append('../bin/')
# import code
import paramtest

# the parameter test function just prints out information about the parameters
paramtest.main(add_jitter=False, scene='loicfile.fits')

Note again the values of `add_jitter` and `scene` and the source location (`kwargs`)

## 5. The parameter dictionary (ParamDict)

The paremeter dictionary has multiple useful features.

### 5.1 Parameter dictionary is case-insensitive

This means unlike a normal dictionary you only have one value describing and characters i.e.:

- PACKAGE_VERSION
- Package_Version
- package_version
- PackAge_VerSion

all link to the same constant:

In [ ]:
print('version = ', params['PACKAGE_VERSION'])
print('version = ', params['Package_Version'])
print('version = ', params['package_version'])
print('version = ', params['PackAge_VerSion'])

### 5.2 Parameter dictionary is locked

In most cases the parameter dictionary `params` should not be added to or modified after the `setup` function. To aid this there is a locking mechanism that will prevent adding to or changing the parameter dictionary:

In [ ]:
params['TEST'] = 2
print('TEST = ', params['TEST'])

In [ ]:
params.lock()

In [ ]:
params['TEST'] = 1
print('TEST = ', params['TEST'])

However in exceptional circumstances it is possible:

In [ ]:
params.set('TEST', value=3, source=__NAME__)
print('TEST = ', params['TEST'])

### 5.3 Find out where a parameter was defined:

In [ ]:
params.sources['SCENE']

In [ ]:
params.sources['PACKAGE_VERSION']

### 5.4 Get information about parameters:

In [ ]:
params.info('SCENE')

In [ ]:
params.info('JITTER_RMS')

In [ ]:
import numpy as np
params.set('LOICARRAY', np.arange(100), source='test')

params.info('LOICARRAY')


### 5.5 Lists and dictionaries from strings

Because config files and arguments may have trouble with inputs such as list and dictionary the parameter file has special ways to open these, these methods are `listp` and `dictp`.

In [ ]:
# lets unlock the parameter dictionary for now
params.unlock()

# lets define a parameter (that may come from the constants file, config file or command line)
# You'll notice the "list" here is not like python but just a python string separated by commas
params['WAVES'] = '400, 500, 600'

To use this 'string list' as a list we have the `listp` method:

In [ ]:
wavelengths = params.listp('WAVES', dtype=float)
print(wavelengths)

The same is true for dictionaries (note be careful with the use of `'` and `"`)

In [ ]:
params['COLOURS'] = "{'red': 'r', 'blue': 'b', 'green': 'g'}"
params['PEOPLE'] = '{"bob":10, "fred":20, "chris": 30}'

In [ ]:
colours = params.dictp('COLOURS')
print(colours)

In [ ]:
people = params.dictp('PEOPLE')
print(people)

In our constants file these would look as follows:

    # define the wavelengths
    WAVES = 400, 500, 600

    # define the colours for plotting
    COLOURS = {'red': 'r', 'blue': 'b', 'green': 'g'}

    # define the people
    PEOPLE = {"bob":10, "fred":20, "chris": 30}


### 5.6 Finding a parameter

There are some useful string methods that are also usable to search for specific keys (useful when there are many constants defined)

In [ ]:
params.startswith('PACK')

In [ ]:
params.contains('JITTER')

In [ ]:
params.endswith('FILE')

In [ ]:
list(params.keys())